In [1]:
import findspark
findspark.init("D:/spark")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [3]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadFromConsoleToDF") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

In [4]:
lines = spark.readStream \
.format("socket") \
.option("host","localhost") \
.option("port", 9999) \
.load()

In [5]:
words = lines.withColumn("column1", f.split(f.col("value"), "\s+")[0]) \
                .withColumn("column2", f.split(f.col("value"), "\s+")[1]) \
                .withColumn("column3", f.split(f.col("value"), "\s+")[2]) \
.drop("value")

In [ ]:
# ncat çalıştırılır

In [ ]:
query = words.writeStream \
.outputMode("append") \
.format("console") \
.start()

In [ ]:
query.awaitTermination()

In [ ]:
query.stop()

# Örnek konsol girdi

    merhaba spark merhaba
    merhaba spark merhaba
    selam selam selam
    selam spark selam
    hi hi hi

# Çıktı

    Batch: 0
    -------------------------------------------
    +-------+-------+-------+
    |column1|column2|column3|
    +-------+-------+-------+
    |merhaba|  spark|merhaba|
    +-------+-------+-------+

    -------------------------------------------
    Batch: 1
    -------------------------------------------
    +-------+-------+-------+
    |column1|column2|column3|
    +-------+-------+-------+
    |merhaba|  spark|merhaba|
    +-------+-------+-------+

    -------------------------------------------
    Batch: 2
    -------------------------------------------
    +-------+-------+-------+
    |column1|column2|column3|
    +-------+-------+-------+
    |  selam|  selam|  selam|
    +-------+-------+-------+

    -------------------------------------------
    Batch: 3
    -------------------------------------------
    +-------+-------+-------+
    |column1|column2|column3|
    +-------+-------+-------+
    |  selam|  spark|  selam|
    +-------+-------+-------+

    -------------------------------------------
    Batch: 4
    -------------------------------------------
    +-------+-------+-------+
    |column1|column2|column3|
    +-------+-------+-------+
    |     hi|     hi|     hi|
    +-------+-------+-------+"